#**STATISTICAL PART**

Created on Monday 04 January 2021  

**Group 5 - Identification of new sources**  

@authors : C.P.M, Y.S., S.B. 

##**1/ Import of library**

In [4]:
!pip install scrapy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from wordcloud import WordCloud
from pandas import DataFrame
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
import pkg_resources,imp
imp.reload(pkg_resources)
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
import pickle
import time
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import json
import random
import scrapy
from scrapy import Selector
from requests import get
from urllib.parse import urlencode
from urllib.parse import urlparse
from datetime import datetime
from datetime import date
from datetime import timedelta

     |████████████████████████████████| 245kB 7.5MB/s 
     |████████████████████████████████| 3.2MB 11.4MB/s 
     |████████████████████████████████| 3.1MB 45.6MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 2.6MB 26.1MB/s 
     |████████████████████████████████| 245kB 49.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=b099a103388f113b70e8fe0e8f718ba090c61fd139415ce88f887242ac34c9df
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=ce65582389b2aa6617570eb9716e0eef5c1946952f6d6fe2e8210240ec86c4ee
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego P

##**2/ File import**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data = pd.read_json('/content/drive/My Drive/df_sources.json')

In [7]:
data.drop(['score'], axis='columns', inplace=True)
data = data.rename(columns = {'URL': 'art_url', 'Query': 'query'}) 

##**3/ Some statistics on the names of sites ...**

1 - Preparation of data

In [8]:
# Step 1 : get the domain (site name) of the site thanks to the url of the article  

def site_name(url: str) -> str :
    """Documentation

        Parameter:
            url: complete url of an article

        Out:
            base_url: name of a website

        """
    site = url.split("://")
    if site[0] == "https" or site[0] == "http":
        name_site = site[1]
    else:
        name_site = site[0]
    tab = name_site.split("/")
    name_site = tab[0]
    TLD = ["fr.","www.","www2.",".org",".fr",".eu",".net",".com"]
    for i in TLD:
        name_site = name_site.replace(i, "")
    return(name_site)

def get_base_url(url: str) -> str:
        """Documentation

        Parameter:
            url: complete url of an article

        Out:
            base_url: base url of a website

        """
        for val in re.finditer(r"(\w)+://[^/]+/", url):
            base_url = val.group(0)
        return base_url

In [9]:
def prepareDF(df): 
    df['src_name'] = [site_name(url) for url in df.art_url]
    df["src_url"] = [get_base_url(url) for url in df.art_url]
    return df
data = prepareDF(data)

In [10]:
# Step 2 : put all the names of the sites in a list ...

list_sites : list = []

for i in range (len(data)):
  name : str = site_name(str(data['art_url'][i]))
  name : str = name.upper() #puts in uppercase 
  name : str = name.replace(".", "_")
  name : str = name.replace(" ","_")
  name : str = name.replace("-","_")
  list_sites.append(name)

In [11]:
# Step 3 : ... or in a dataframe, as needed

df_sites = DataFrame(list_sites,columns=['src_name'])

In [12]:
# function that retrieves the url of the site thanks to the url of the article article

def lienRacine(lienArticle) :
  split = lienArticle.split('/')
  return (split[0]+'//'+split[2])

def splitPath(lien) :
  listePath = lien.split('/')[3:]
  for mot in range(len(listePath)) :
    if listePath[mot] =='' :
      listePath.remove('')
  return listePath

## **4/ Relevance score :**

In [13]:
#### Popularité ####
def countSite(df):
  return df.groupby('src_name').count().reset_index()

def popularite(df):
    """
    Parameters
    ----------
    df : DataFrame
        Contains : art_url, src_name, src_url.

    Returns
    -------
    DataFrame
        Contains : src_name, popularity score.

    """
    df1 = countSite(df)
    df1["popularity"] = [x/np.max(df1.art_url) for x in df1.art_url]
    df_final = df1[['src_name','popularity']]
    return df_final
df_resultat = popularite(data) #the modified dataframe is saved 

**2 -** Estimation of the relevance by taking the **words in common between those of the query, and those present in the title and in the Google summary** of the article.

In [14]:
# Step 1 : Create the cleanup to get the root, remove punctuation and empty words

def cleandesc(desc):
    sent = desc
    sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
    Porter = SnowballStemmer('french')
    sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
    sent = " ".join(sent.split())
    return sent
start_time = time.time()

In [15]:
# Step 2 : Apply the cleaning function is applied 

data['title'] = [cleandesc(x.title) for x in data.itertuples()]
data['resume'] = [cleandesc(x.resume) for x in data.itertuples()]
data['query'] = [cleandesc(x.query) for x in data.itertuples()]
end_time = time.time()
print("total time : {} mn".format((end_time-start_time)/60))

total time : 0.02756366729736328 mn


In [16]:
# Step 3 : to do the function that will determine the relevance of the article according to the request 

data['cat_pertinence'] = ''
data['common_words'] = ''
data['relevance_query']=0.00
relevance_query : list = [] #list that will store the relevance score,
#pertinence_j = [] #list that will store the number of the line concerned
def common_query_words(df) :
  df_relevance = pd.DataFrame(columns=['nb_row','score']).set_index('nb_row') #creation of a df allowing to have the score for each line,
  for j in range (len(df)): # for each line of the df,
    innov : int = 0 #initialization of a variable to count the number of words in the innovation lexicon 
    gest : int = 0 #...the same for the management lexicon
    separator_or : list = list(df['query'][j].split(' or ')) #we store all pairs of the request in a separator_or list 
    relevance_listing : list = [] #list to store the relevance scores for each of the couples
    list_find : list = [] #list to store words in common for each line,
    for k in range (len(separator_or)): #for each of these couples,
      nb_present : int = 0 #count the number of words in common
      separator_and : list = list(separator_or[k].split(' and ')) #we store all the words of the couple of the query in a separator_and list 
      for i in range (len(separator_and)): #for all the words in the query,
        if (df['title'][j].find(separator_and[i]) != -1) : #we look for it in the title, and if it's there... 
          if (i == 0): #if it's the first member of the couple, we've found a word of innovation 
            innov += 1
          else : #If not, a word of management 
            gest += 1
          nb_present = nb_present + 1 #increments the number of words in the query found.  
          list_find.append(separator_and[i]+";")
        if (df['resume'][j].find(separator_and[i]) != -1) : #then we look for it in the summary, and if it's there... 
          if (i == 0):
            innov += 1
          else :
            gest += 1
          nb_present += 1
          list_find.append(separator_and[i]+";") #... we put it in a list
          list_find = list(set(list_find))
      relevance_query : float = (nb_present / len(separator_and))*100 #we calculate the relevance score nb of words found / nb of total words in the query
      relevance_listing.append(relevance_query)
      str = ' '.join(list_find) #...
    df['common_words'][j]=str[:-1] #...we add the words found in the df
    df_relevance = df_relevance.append({'nb_row': j}, ignore_index=True)
    df_relevance['score'][j] = max(relevance_listing) #for each line, we take the best relevance of a couple,
    df['relevance_query'][j] = df_relevance['score'][j]
    nb = df_relevance['score'][j]
    #and we categorize according to the score obtained :
    if (nb>=100) : #all the words of at least one couple are found,
      df['cat_pertinence'][j] = 'I&G'
    else :
      if (innov >= 1 and gest >= 1) : #words of innovation and gestion are found but not in the same couple,
        df['cat_pertinence'][j] = 'I&G but not from the same couple'
      elif (innov >= 1 and gest == 0) : #at least one word of innovation is found, but no gestion,
        df['cat_pertinence'][j] = 'I'
      elif (innov == 0 and gest >= 1): #at least one word of management is found but no innovation, 
        df['cat_pertinence'][j] = 'G'
      else :
        df['cat_pertinence'][j] = 'None' #no word is found 
  return(df)

In [17]:
def result_score_def (df):
  df_relevance_query = common_query_words(df)
  df_relevance_query['src_name'] = ''
  for i in range (len(df_relevance_query)) :
    df_relevance_query['src_name'][i] = site_name(df_relevance_query['art_url'][i])
  df_result_score = df_relevance_query.groupby('src_name')['relevance_query'].mean()
  return (df_result_score)

df_result_score = result_score_def(data)
df_result_score = df_result_score.reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to b

In [20]:
df_result_score['relevance_query'] = df_result_score['relevance_query']/(max(df_result_score['relevance_query']))

In [22]:
fusion = pd.merge(df_resultat, df_result_score, how="right", left_on="src_name", right_on="src_name")

**3 -** Estimation of relevance by the **position of the article in the crawl**

In [24]:
def score_rank(df) :
  for i in range(df.shape[0]):
    df['score_rank'] = 1 - df['position']/df[df['query'] == df['query'].iloc[i]].shape[0]
    df_result_rank = df.groupby('src_name')['score_rank'].mean()
    return (df_result_rank)
df_result_rank = score_rank(data)
df_result_rank = df_result_rank.reset_index()

In [25]:
# All relevant measurements are then merged into a single dataframe.
fusion_result = pd.merge(fusion, df_result_rank, how="right", left_on="src_name", right_on="src_name")
fusion_result['score_mean']=0.0

In [27]:
for i in range (len(fusion_result)):
  fusion_result['score_mean'][i] = fusion_result['popularity'][i]*0.2 + fusion_result['relevance_query'][i]*0.4 + fusion_result['score_rank'][i]*0.4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
fusion_result

,src_name,popularity,relevance_query,score_rank,score_mean
0,actionfirst,0.333333,1.0,0.373737,0.616162
1,actufinance,0.333333,0.0,0.020202,0.074747
2,aijobs.tech,0.333333,1.0,0.949495,0.846465
3,alliedmarketresearch,0.333333,0.0,0.000000,0.066667
4,amazing.dev,0.333333,0.5,0.676768,0.537374
...,...,...,...,...,...
186,whatis.techtarget,0.333333,0.5,0.030303,0.278788
187,wired.jp,0.333333,0.5,0.434343,0.440404
188,wisdom.nec,0.333333,0.0,0.535354,0.280808
189,worldbrainmapping,0.333333,0.5,0.616162,0.513131
